#### Librerias

In [2]:
import scipy.io
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import datetime
import csv
import ExtracurricularSinc.funciones as mifu

###### Voy a probar hacer un primer analisis en frecuencias de la señal usando fourier:

In [4]:
sujeto = 1
prediccion = 'estimulo'

datos = mifu.extraer(sujeto)
datos_ordenados = mifu.ordenar(datos)
datos_normalizados = mifu.normalizar(datos_ordenados)

In [19]:
x = np.array([1,2,3,4,5,6])
random.shuffle(x, 11)
x

TypeError: Random.shuffle() takes 2 positional arguments but 3 were given

In [26]:
print(datos.shape)
print(datos_ordenados[0].shape)



(688, 24579)
(688, 24576)


(63, [70, 63, 67, 66, 65, 62, 61, 58, 60, 58, 58])